# Identifying (volatility-)regimes in the EUR/USD spot exchange rate using clustering algorithms: An Oil and Gas Perspective on Parity Conditions.
Seminar in Applied Financial Economics: Applied Econometrics of FX Markets - Prof. Dr. Reitz
<br>
**Josef Fella and Robert Hennings**
<br>
Christian Albrechts University of Kiel
<br>
*josef.fella@stu.uni-kiel.de and robert.hennings@stu.uni-kiel.de*
<br>
GitHub: https://github.com/RobertHennings/Seminar
<br>
Kiel - 14.11.2025


**Group 08**
<br>
Josef Fella: stu245231, Quantitative Finance
<br>
Robert Hennings: stu236320, Quantitative Finance

## Outline
1. Research Hypothesis
<br>
1.1 Energy Commodity Price Shocks: The Pass-Through Effect and Implications for Monetary Policy
<br>
1.2 Research Hypothesis
<br>
2. Theoretical Framework
<br>
2.1 A simple model of exchange rates and commodity prices
<br>
2.2 Theoretical Framework
<br>
3. Model Results
<br>
3.1 Regime identification - Model comparison and selection
<br>
4. Conclusion and Discussion
<br>
4.1 Seminar Project Summary
<br>
4.2 Seminar Project Limitations
<br>
4.3 Future Research
<br>
5. Appendix
<br>
5.1 Abbreviations
<br>
5.2 Systematic Literature Overview: Main Approaches
<br>
5.3 Figures and Tables
<br>
5.3 Definitions and Data
<br>

## Short description of the notebook contents
The contents of this Jupyter notebook produce all the included graphs of the seminar presentation. It is based on the file seminar_graphs.py, that includes all the detailed data generating procedures, what have been skipped here in order to not cause confusion, if data can't be appropriatley loaded from the various sources due to a number of different potential reasons.

## Initial Settings

Import dependencies/packages:

In [ ]:
import os
import json
import logging
import pandas as pd
import numpy as np
import sklearn as sklearn

Set global config settings:

**!!!!CHANGE WORKING DIRECTORY HERE!!!!**

In [ ]:
SEMINAR_PATH = r"/Users/josef/Desktop/Seminar/full_submission/code"
# Example: r"/Users/Robert_Hennings/Uni/Master/Seminar"

In [ ]:
SEMINAR_CODE_PATH = rf"{SEMINAR_PATH}/src/seminar_code"
MODELS_PATH = rf"{SEMINAR_CODE_PATH}/models"
FIGURES_PATH = rf"{SEMINAR_PATH}/reports/figures"
TABLES_PATH = rf"{SEMINAR_PATH}/reports/tables"
DATA_PATH = rf"{SEMINAR_PATH}/data"
PRESENTATION_DATA = rf"{SEMINAR_PATH}/data"
NUM_YEARS_INTERVAL_X_AXIS = 5
RENDERER = "vscode+browser"
CAU_COLOR_SCALE = ["#9b0a7d", "grey", "black", "darkgrey", "lightgrey"]

# Change working directory to seminar code path
print(os.getcwd())
os.chdir(SEMINAR_CODE_PATH) # <- needed to be able to import the ModelObject class from utils
print(os.getcwd())

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Import custom written model evaluation functions
from utils.evaluation import get_recoded_predicted_labels_df

# Import data graphing class
from utils.data_grapher import DataGraphing

# Instantiate data plot class
data_graphing_instance = DataGraphing()

## 0.Intro: The PPP puzzle and Commodity Currencies

In [ ]:
# Load exchange rates data
file_name = r"chap_00_exchange_rates_ppp_deviations.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
exchange_rates_df = pd.read_excel(full_file_path, index_col=0)
print(f"Loaded exchange rates data:\n{exchange_rates_df.head(n=10)}")

In [ ]:
# Calculate effective exchange rates based on PPP-values
effective_exchange_rates_df = pd.DataFrame()
effective_exchange_rates_df["GBPREAL"] = np.log(exchange_rates_df["United Kingdom"]) - np.log(exchange_rates_df["United States"])
effective_exchange_rates_df["JPYREAL"] = np.log(exchange_rates_df["Japan"]) - np.log(exchange_rates_df["United States"])
effective_exchange_rates_df["CHFREAL"] = np.log(exchange_rates_df["Switzerland"]) - np.log(exchange_rates_df["United States"])
effective_exchange_rates_df["EUROREAL"] = np.log(exchange_rates_df["Euro Area (EA)"]) - np.log(exchange_rates_df["United States"])

# Prepare data and settings for graphing
data = effective_exchange_rates_df.copy()
variables = data.columns.tolist()
secondary_yaxis_variables = []
title = f"Monthly deviations of USD Spot Rate from PPP-values over the time: {data.index[0].year} - {data.index[-1].year}"
x_axis_title = "Time"
y_axis_title = "Deviations of USD Spot Rate from PPP-values (log)"
secondary_yaxis_title = ""
color_mapping = {
    'GBPREAL': "grey",
    'JPYREAL': "black",
    'CHFREAL': "#9b0a7d",
    'EUROREAL': "darkgrey"
}

# Generate figure
fig_deviations_from_ppp = data_graphing_instance.get_fig_deviations_ppp(
        data=data,
        variables=variables,
        secondary_y_variables=secondary_yaxis_variables,
        title="",
        secondary_y_axis_title=secondary_yaxis_title,
        color_discrete_sequence=CAU_COLOR_SCALE,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        color_mapping_dict=color_mapping,
        save_fig=False,
        file_name="chap_00_deviations_of_usd_spotrates_from_ppp_values",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )

# Show figure
fig_deviations_from_ppp.show()

## 1.Research Hypothesis

### 1.1 Energy Commodity Price Shocks: The Pass-Through Effect and implications for Monetary Policy

#### Energy Price Contributions to Inflation - USA

In [ ]:
# Load US inflation contribution data
file_name = r"chap_01_us_inflation_contribution_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
data_us_df = pd.read_excel(full_file_path, index_col=0)
print(f"Loaded US inflation contribution data:\n{data_us_df.head(n=10)}")

In [ ]:
# Settings for US CPI inflation decomposition graph
series_dict_mapping = {
    'Headline': 'CPIAUCSL',
    'Core CPI': 'CPILFESL',
    'Energy': 'CPIENGSL',
    'Food': 'CPIUFDSL',
}
weights = {
    'Core CPI': 1 - 0.136 - 0.072,  # Core (everything except food & energy)
    'Energy': 0.072,
    'Food': 0.136,
}
start_date = '2000-01-01'
end_date = '2024-12-31'
headline_id = 'CPIAUCSL'  


def percent_change(series):
    return 100 * (series.iloc[-1] / series.iloc[-13] - 1)

# Compute 12-month pct. change (annual inflation): (P_t / P_{t-12} - 1) * 100
data_pct = data_us_df.pct_change(periods=12) * 100

# Compute contributions: weight × component inflation
contributions = pd.DataFrame(index=data_pct.index)
for col in weights:
    contributions[col] = data_pct[col] * weights[col]

# Only these bar components
bar_components = weights.keys()
contributions = contributions[bar_components].dropna()

# Sort contributors by value for each period (row), biggest on bottom
contributions_sorted = pd.DataFrame(
    np.sort(contributions[bar_components].values, axis=1)[:, ::-1],  # sort and reverse for descending
    index=contributions.index,
    columns=[f"{i+1}" for i in range(len(bar_components))]
)

# Next, get the names for the columns for legend and color mapping
def get_sorted_labels(row):
    # Sort values for the row, get corresponding labels
    vals = row.values
    labels = [x for _, x in sorted(zip(vals, bar_components), reverse=True)]
    return labels

labels_sorted = contributions[bar_components].apply(get_sorted_labels, axis=1)

variables = list(data_us_df.columns)
energy_color = "#9b0a7d"
cpi_color = "red"  # Fixed color for the "World" category

color_mapping = {
    'Core CPI': "grey",
    'Energy': energy_color,
    'Food': "lightgrey",
    'Headline': cpi_color,
}
title = f"US CPI: Headline and component contributions over the time: {contributions.index[0].year} - {contributions.index[-1].year}"
x_axis_title = "Time"
y_axis_title = 'Contribution to annual inflation (%)'

# Generate figure
fig_inflation_decomp_usa = data_graphing_instance.get_fig_inflation_contribution_usa(
        data=contributions,
        data_pct=data_pct,
        cpi_color=cpi_color,
        variables=list(bar_components),
        title="",
        secondary_y_variables=[],
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        color_mapping_dict=color_mapping,
        save_fig=False,
        file_name="chap_02_us_cpi_inflation_decomposition",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )

# Show figure
fig_inflation_decomp_usa.show()

#### Energy Price Contributions to Inflation - EU area

In [ ]:
# Read in Euro Area data
file_name = r"chap_01_eu_inflation_contribution_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
data = pd.read_excel(full_file_path, index_col=0)
print(f"Loaded Euro Area inflation contribution data:\n{data.head(n=10)}")

In [ ]:
# Define weights
weights = {
    "Food": 0.172,      # 17.2%
    "Energy": 0.098,    # 9.8%
    "Core CPI": 1 - 0.172 - 0.098,  # remainder
}
data = data.resample('M').last()  # If needed, ensure monthly end-of-period alignment

# Here, each value is ALREADY YoY % change. So simply multiply by weights for contributions:
contributions = pd.DataFrame(index=data.index)
for col in weights:
    contributions[col] = data[col] * weights[col]

bar_components = data.columns.difference(['Headline'])
contributions = contributions[bar_components].dropna()
# Sort contributors by value for each period (row), biggest on bottom
contributions_sorted = pd.DataFrame(
    np.sort(contributions[bar_components].values, axis=1)[:, ::-1],  # sort and reverse for descending
    index=contributions.index,
    columns=[f"{i+1}" for i in range(len(bar_components))]
)
labels_sorted = contributions[bar_components].apply(get_sorted_labels, axis=1)


variables = list(contributions.columns)
contributions["Headline"] = data["Headline"]


energy_color = "#9b0a7d"
cpi_color = "red"  # Fixed color for the "World" category

color_mapping = {
    'Core CPI': "grey",
    'Energy': energy_color,
    'Food': "lightgrey",
    'Headline': cpi_color,
}
title = f"EU Area CPI: Headline and component contributions over the time: {contributions.index[0].year} - {contributions.index[-1].year}"
x_axis_title = "Time"
y_axis_title = 'Contribution to annual inflation (%)'

fig_inflation_decomp_euro_area = data_graphing_instance.get_fig_inflation_contribution_euro_area(
        data=contributions,
        cpi_color=cpi_color,
        variables=variables,
        secondary_y_variables=[],
        title="",
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        color_mapping_dict=color_mapping,
        save_fig=False,
        file_name="chap_02_eu_area_cpi_inflation_decomposition",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_inflation_decomp_euro_area.show()

#### Rolling Volatility of Exchange Rates and Energy Commodity Prices

In [ ]:
file_name = r"chap_01_daily_exchange_rate_oil_log_diff_vola_normalized_crisis_periods_highlighted.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
data_log_first_diff = pd.read_excel(full_file_path, index_col=0)
print(f"Loaded daily exchange rate, oil log diff volatility data:\n{data_log_first_diff.head(n=10)}")

file_name = r"crisis_periods_dict.json"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"

with open(full_file_path, "r") as f:
    crisis_periods_dict = json.load(f)

In [ ]:
data_log_diff_normalized = (data_log_first_diff - data_log_first_diff.min()) / (data_log_first_diff.max() - data_log_first_diff.min())
title=f"Daily normalized EUR/USD spot exchange rate, oil and gas log first differences volatility with highlighted crisis periods over the time: {data.index[0].year} - {data.index[-1].year}"
x_axis_title="Time"
y_axis_title="Volatility of log first differences (normalized)"


fig_crisis_periods_highlighted = data_graphing_instance.get_fig_crisis_periods_highlighted(
    data=data_log_diff_normalized,
    crisis_periods_dict=crisis_periods_dict,
    variables=data_log_diff_normalized.columns,
    secondary_y_variables=[],
    recession_shading_color="rgba(155, 10, 125, 0.3)",
    title="",
    secondary_y_axis_title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    color_mapping_dict={
        'EUR/USD': "grey",
        'WTI Oil': "black",
        'Natural Gas': "#9b0a7d",
    },
    num_years_interval_x_axis=5,
    showlegend=False,
    save_fig=False,
    file_name="chap_02_daily_exchange_rate_oil_log_diff_vola_normalized_crisis_periods_highlighted",
    file_path=FIGURES_PATH,
    width=1200,
    height=800,
    scale=3
    )
# Show the figure
fig_crisis_periods_highlighted.show()

## 4.Model Results

### 4.1 Regime identification - Model comparison and selection

#### Model comparison and selection

In [ ]:
# Load model comparison data
file_name = r"chap_04_unique_df_full.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
unique_df = pd.read_excel(full_file_path)
print(f"Loaded model comparison data:\n{unique_df.head(n=10)}")

In [ ]:
title="Model comparison using the silhouette score (1 being best, 0 indicating overlapping clusters, -1 being worst) for various regime identification model configurations"

# Generate model comparison bar plot
fig_model_comp_bar_plot = data_graphing_instance.get_model_comparison_bar_plot(
    data=unique_df,
    evaluation_score_col_name="silhouette_score",
    title="",
    x_axis_title="Feature names",
    y_axis_title="Silhouette score (1 best - 0 overlapping clusters - -1 worst)",
    color_mapping_dict={
        "KMeans": "black",
        "AgglomerativeClustering": "darkgrey",
        "DBSCAN": "blue",
        "MeanShift": "red",
        "MarkovRegression": "#9b0a7d",
        "GaussianMixture": "lightgrey",
        "Birch": "pink",
        "AffinityPropagation": "gray",
        "OPTICS": "olive",
        "MiniBatchKMeans": "#00677c"
    },
    save_fig=False,
    file_name="model_comparison_bar_plot",
    file_path=FIGURES_PATH,
    showlegend=True,
    textfont_size=8.5,
    width=1400,
    height=800,
    scale=3
    )

# Show figure
fig_model_comp_bar_plot.show()

#### Predicted Regimes - Evolution over time with highlighted Crisis Periods

In [ ]:
file_name = r"chap_04_predicted_labels_df.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
predicted_labels_df = pd.read_excel(full_file_path, index_col=0)
print(predicted_labels_df.head(n=30))

In [ ]:
file_name = r"chap_01_daily_exchange_rate_oil_log_diff_vola_normalized_crisis_periods_highlighted.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
rolling_std_df = pd.read_excel(full_file_path, index_col=0)
print(rolling_std_df.head(n=30))

In [ ]:
# Now also map the regimes across the models correctly, 
# in that we assume the high vola regime is encoded as 1 and the low vola regime is a 0:
# - KMeans is already correctly encoded
# - AgglomerativeClustering needs to be encoded
# - MeanShift is also correctly encoded
# - MarkovRegression needs to be encoded

# Recoding predicted labels accordingly
predicted_labels_df = get_recoded_predicted_labels_df(
    predicted_labels_df=predicted_labels_df,
    label_mapping_dict={0: 1, 1: 0},
    column_names_list=["AgglomerativeClustering", "MarkovRegression"]
    )

# Normalize rolling std. between 0 and 1 for better visualization
rolling_std_df = (rolling_std_df - rolling_std_df.min()) / (rolling_std_df.max() - rolling_std_df.min())

graphing_df = pd.concat(
    [rolling_std_df, predicted_labels_df],
    axis=1
)

file_name = r"crisis_periods_dict.json"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"

with open(full_file_path, "r") as f:
    crisis_periods_dict = json.load(f)


custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(graphing_df.columns)+1)/2),
    lightening_factor=0.8,
)

# Extract onlyhex color codes from created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(graphing_df.columns.tolist())}

for column in graphing_df.columns:
    if not column == "EUR/USD":
        if graphing_df[column].max() > 1 or graphing_df[column].min() <0 or (graphing_df[column].dropna() == 0.0).all():
            print(f"Removing column: {column}")
            graphing_df = graphing_df.drop(columns=[column])
# graphing_df = graphing_df.drop(["WTI Oil", "Nat Gas"], axis=1)


# Generate figure with crisis periods highlighted
title=f"Resulting predicted model regimes for various variants with highlighted crisis periods over the time: {graphing_df.index[0].year} - {graphing_df.index[-1].year}",
fig_crisis_periods_highlighted = data_graphing_instance.get_fig_crisis_periods_highlighted(
    data=graphing_df.dropna(),
    crisis_periods_dict=crisis_periods_dict,
    variables=["EUR/USD"],
    secondary_y_variables=graphing_df.columns[1:].tolist(),
    recession_shading_color="rgba(155, 10, 125, 0.3)",
    title="",
    secondary_y_axis_title="Predicted model regimes (N=2)",
    x_axis_title="Date",
    y_axis_title="Normalized 30-day rolling volatility of Spot exchange rate EUR/USD, WTI Oil and Nat Gas (log first differences)",
    color_mapping_dict=color_mapping,
    num_years_interval_x_axis=5,
    showlegend=True,
    save_fig=False,
    file_name="predicted_model_regimes_with_crisis_periods_highlighted",
    file_path=FIGURES_PATH,
    width=1200,
    height=800,
    scale=3
    )

# Show figure
fig_crisis_periods_highlighted.show()

#### Predicted Regimes - Overlapping Comparison

In [ ]:
file_name = r"chap_04_unique_df_full.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
unique_df = pd.read_excel(full_file_path)
print(unique_df.head(n=30))

In [ ]:
# Generate model regime counts bar plot
fig_model_regime_counts_bar_plot = data_graphing_instance.get_model_comparison_regime_counts_bar_plot(
        data=unique_df,
        evaluation_score_col_name="silhouette_score",
        title="",
        counts_display="relative",
        color_class_0="grey",
        color_class_1="#9b0a7d",
        x_axis_title="Feature names",
        y_axis_title="Relative Regime Counts (Class 0: lower stack, Class 1: upper stack)",
        color_mapping_dict={
        "KMeans": "black",
        "AgglomerativeClustering": "darkgrey",
        "DBSCAN": "blue",
        "MeanShift": "red",
        "MarkovRegression": "#9b0a7d",
        "GaussianMixture": "lightgrey",
        "Birch": "pink",
        "AffinityPropagation": "gray",
        "OPTICS": "olive",
        "MiniBatchKMeans": "#00677c"
        },
        save_fig=False,
        file_name="model_comparison_regime_count_bar_plot",
        file_path=FIGURES_PATH,
        showlegend=True,
        textfont_size=8.5,
        width=1400,
        height=800,
        scale=3
        )

# Show figure
fig_model_regime_counts_bar_plot.show()

#### Predicted Regimes - Overlapping Comparison with theoretical crisis periods

In [ ]:
# Load combined model coefficients data
file_name = r"chap_04_unique_df_full.xlsx" # comes from file model_benchmark.py
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
unique_df = pd.read_excel(full_file_path)

# Generate model comparison bar plot for high volatility percentage
fig_model_comp_bar_plot = data_graphing_instance.get_model_comparison_bar_plot(
    data=unique_df,
    evaluation_score_col_name="High Volatility Percentage",
    title="",
    x_axis_title="Feature names",
    y_axis_title="High Volatility Percentage",
    color_mapping_dict={
        "KMeans": "black",
        "AgglomerativeClustering": "darkgrey",
        "DBSCAN": "blue",
        "MeanShift": "red",
        "MarkovRegression": "#9b0a7d",
        "GaussianMixture": "lightgrey",
        "Birch": "pink",
        "AffinityPropagation": "gray",
        "OPTICS": "olive",
        "MiniBatchKMeans": "#00677c"
    },
    save_fig=False,
    file_name="chap_07_predicted_model_regimes_rel_share_overlap_theo_crisis_regimes",
    file_path=FIGURES_PATH,
    showlegend=True,
    textfont_size=8.5,
    display_red_line_at_y_equals_1=False,
    width=1400,
    height=800,
    scale=3
    )

# Show figure
fig_model_comp_bar_plot.show()

07 - Model Results - Predicted Regimes - UIP estimation - benchmark models

In [ ]:
# Load combined model coefficients data
file_name = r"chap_04_combined_model_coefs_df.xlsx" # comes from file model_benchmark.py
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
combined_model_coefs_df = pd.read_excel(full_file_path, index_col=0)

columns_keep = ["coef", "std_err", "ci_lower", "ci_upper", "t", "model_file_name"]
combined_model_coefs_df = combined_model_coefs_df[columns_keep]

# Generate coefficients with confidence intervals plot
fig = data_graphing_instance.plot_coefs_with_ci(
    data=combined_model_coefs_df.reset_index(),
    model_col="model_file_name",
    coef_col="coef",
    ci_lower_col="ci_lower",
    ci_upper_col="ci_upper",
    t_col="t",
    param_index_name="param",
    title="",
    marker_color="#9b0a7d",
    group_width=0.4,
    marker_size=11,
    save_fig=False,
    file_name="chap_07_uip_estimation_benchmark_models",
    file_path=FIGURES_PATH,
    width=1400,
    height=800,
    scale=3
    )

# Show figure
fig.show()

07 - Model Results - Predicted Regimes - UIP estimation - identiﬁed regimes

In [ ]:
# Load UIP identified regimes results data
file_name = r"chap_04_uip_identified_regimes_results_df.xlsx" # from file model_benchmark.py
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
uip_identified_regimes_results_df = pd.read_excel(full_file_path)

# Generate coefficients with confidence intervals plot
fig = data_graphing_instance.plot_coefs_with_ci(
    data=uip_identified_regimes_results_df,
    model_col="model_name",
    coef_col="coef",
    ci_lower_col="ci_lower",
    ci_upper_col="ci_upper",
    t_col="t",
    param_index_name="param",
    title="",
    marker_color="#9b0a7d",
    group_width=0.8,
    marker_size=9,
    save_fig=False,
    file_name="chap_07_uip_estimation_identified_regimes",
    file_path=FIGURES_PATH,
    width=1400,
    height=800,
    scale=3
    )

# Show figure
fig.show()

## 5 - Appendix

### 5.1 Figures and Tables

In [ ]:
# Load data for oil and gas consumption and production
file_name = r"chap_06_oil_consumption_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
oil_consumption_df = pd.read_excel(full_file_path)

file_name = r"chap_06_oil_production_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
oil_production_df = pd.read_excel(full_file_path)

file_name = r"chap_06_gas_consumption_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
gas_consumption_df = pd.read_excel(full_file_path)

file_name = r"chap_06_gas_production_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
gas_production_df = pd.read_excel(full_file_path)

#### Oil: Global Production and Consumption over time

In [ ]:
variables = ["United States", "China", "India", "Japan", "Russia", "Saudi Arabia", "South Korea", "Canada", "Brazil", "European Union (27)"]
secondary_y_variables = ["World"]
# Figure out the common start date for a shared x-axis
start_year_consumption = oil_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year_consumption = oil_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()
start_year_production = oil_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year_production = oil_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()

oil_start_year = max(start_year_consumption, start_year_production)

start_year = oil_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year = oil_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()
title = f"Yearly oil consumption by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
y_axis_title = "Oil consumption (in terawatt-hours)"
secondary_y_axis_title = "World oil consumption (in terawatt-hours)"

custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(variables)+1)/2),
    lightening_factor=0.8,
)
# Extract only the hex color codes from the created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
world_color = "red"  # Fixed color for the "World" category
# Create a mapping for the colors
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(variables)}
color_mapping["World"] = world_color  # Assign the fixed color for "World"

oil_consumption_df["Year"] = pd.to_datetime(oil_consumption_df["Year"], format="%Y")
oil_consumption_df = oil_consumption_df.pivot(index="Year", columns="Entity", values="oil_consumption_twh")
oil_consumption_df = oil_consumption_df[variables + secondary_y_variables]

# See if the overall pricture get better when we restrict the data to only the relative share of the US
oil_consumption_df_usa = oil_consumption_df[["United States", "World"]]
# Calculate the relative share of the US in world oil consumption
oil_consumption_df_usa["United States Share of World"] = oil_consumption_df_usa["United States"] / oil_consumption_df_usa["World"]

fig_oil_consumption = data_graphing_instance.get_fig_consumption_production_oil_gas(
        data=oil_consumption_df,
        variables=variables,
        secondary_y_variables=secondary_y_variables,
        title=title,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        secondary_y_axis_title=secondary_y_axis_title,
        color_mapping_dict=color_mapping,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        save_fig=False,
        file_name="chap_06_yearly_oil_consumption_by_country",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_oil_consumption.show()

In [ ]:
oil_production_df = oil_production_df.copy()

variables = ["United States", "Russia", "Saudi Arabia", "Canada", "Iran", "China", "Brazil", "Norway", "European Union (27)"]
secondary_y_variables = ["World"]

oil_production_df = oil_production_df[oil_production_df.Year >= oil_start_year]
start_year = oil_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year = oil_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()

title = f"Yearly oil production by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
y_axis_title = "Oil production (in terawatt-hours)"
secondary_y_axis_title = "World oil production (in terawatt-hours)"

custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(variables)+1)/2),
    lightening_factor=0.8,
)
# Extract only the hex color codes from the created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
world_color = "red"  # Fixed color for the "World" category

# Create a mapping for the colors
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(variables)}
color_mapping["World"] = world_color  # Assign the fixed color for "World"

oil_production_df["Year"] = pd.to_datetime(oil_production_df["Year"], format="%Y")
oil_production_df = oil_production_df.pivot(index="Year", columns="Entity", values="oil_production_twh")
oil_production_df = oil_production_df[variables + secondary_y_variables]

# Generate figure
fig_oil_production = data_graphing_instance.get_fig_consumption_production_oil_gas(
        data=oil_production_df,
        variables=variables,
        secondary_y_variables=secondary_y_variables,
        title=title,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        secondary_y_axis_title=secondary_y_axis_title,
        color_mapping_dict=color_mapping,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        save_fig=False,
        file_name="chap_01_yearly_oil_production_by_country",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )

# Show figure
fig_oil_production.show()

In [ ]:
subplot_titles=(
    f"Yearly oil consumption by country (in terawatt-hours) over the time: {start_year} - {end_year}",
    f"Yearly oil production by country (in terawatt-hours) over the time: {start_year} - {end_year}")
title = f"Yearly oil consumption and production by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
secondary_y_variable = "World"

fig_oil_consumption_production_combine = data_graphing_instance.get_combined_production_consumption_graph(
        subplot_titles=list(subplot_titles),
        title="",
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        x_axis_title=x_axis_title,
        secondary_y_variable=secondary_y_variable,
        rows=2,
        cols=1,
        shared_xaxes=False,
        vertical_spacing=0.25,
        specs=[[{"secondary_y": True}], [{"secondary_y": True}]],
        fig_production=fig_oil_production,
        fig_consumption=fig_oil_consumption,
        save_fig=False,
        file_name="chap_01_yearly_oil_consumption_production_combined_graph",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_oil_consumption_production_combine.show()

#### Gas: Global Production and Consumption over time

In [ ]:
variables = ["United States", "China", "Russia", "Iran", "Canada", "Australia", "Saudi Arabia", "European Union (27)"]
secondary_y_variables = ["World"]
# Figure out the common start date for a shared x-axis
start_year_consumption = gas_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year_consumption = gas_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()
start_year_production = gas_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year_production = gas_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()
gas_start_year = max(start_year_consumption, start_year_production)

start_year = gas_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year = gas_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()

title = f"Yearly gas consumption by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
y_axis_title = "Gas consumption (in terawatt-hours)"
secondary_y_axis_title = "World gas consumption (in terawatt-hours)"

custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(variables)+1)/2),
    lightening_factor=0.8,
)
# Extract only the hex color codes from the created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
world_color = "red"  # Fixed color for the "World" category

# Create a mapping for the colors
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(variables)}
color_mapping["World"] = world_color  # Assign the fixed color for "World"

gas_consumption_df["Year"] = pd.to_datetime(gas_consumption_df["Year"], format="%Y")
gas_consumption_df = gas_consumption_df.pivot(index="Year", columns="Entity", values="gas_consumption_twh")
gas_consumption_df = gas_consumption_df[variables + secondary_y_variables]

fig_gas_consumption = data_graphing_instance.get_fig_consumption_production_oil_gas(
        data=gas_consumption_df,
        variables=variables,
        secondary_y_variables=secondary_y_variables,
        title=title,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        secondary_y_axis_title=secondary_y_axis_title,
        color_mapping_dict=color_mapping,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        save_fig=False,
        file_name="chap_06_yearly_gas_consumption_by_country",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_gas_consumption.show()

In [ ]:
variables = ["United States", "China", "Iran", "Canada", "Saudi Arabia", "Mexico", "European Union (27)"]
secondary_y_variables = ["World"]

start_year = gas_start_year
end_year = gas_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()

gas_production_df = gas_production_df[gas_production_df.Year >= gas_start_year]

title = f"Yearly gas production by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
y_axis_title = "Production (in terawatt-hours)"
secondary_y_axis_title = "World gas production (in terawatt-hours)"


custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(variables)+1)/2),
    lightening_factor=0.8,
)
# Extract only the hex color codes from the created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
world_color = "red"  # Fixed color for the "World" category

# Create a mapping for the colors
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(variables)}
color_mapping["World"] = world_color  # Assign the fixed color for "World"

gas_production_df["Year"] = pd.to_datetime(gas_production_df["Year"], format="%Y")
gas_production_df = gas_production_df.pivot(index="Year", columns="Entity", values="gas_production_twh")
gas_production_df = gas_production_df[variables + secondary_y_variables]

fig_gas_production = data_graphing_instance.get_fig_consumption_production_oil_gas(
        data=gas_production_df,
        variables=variables,
        secondary_y_variables=secondary_y_variables,
        title=title,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        secondary_y_axis_title=secondary_y_axis_title,
        color_mapping_dict=color_mapping,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        save_fig=False,
        file_name="chap_06_yearly_gas_production_by_country",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_gas_production.show()

In [ ]:

subplot_titles=(f"Yearly gas consumption by country (in terawatt-hours) over the time: {start_year} - {end_year}",
                f"Yearly gas production by country (in terawatt-hours) over the time: {start_year} - {end_year}")
title = f"Yearly gas consumption and production by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
secondary_y_variable = "World"


# Generate combined consumption and production figure
fig_gas_consumption_production_combine = data_graphing_instance.get_combined_production_consumption_graph(
        subplot_titles=subplot_titles,
        title="",
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        x_axis_title=x_axis_title,
        secondary_y_variable=secondary_y_variable,
        rows=2,
        cols=1,
        shared_xaxes=False,
        vertical_spacing=0.25,
        specs=[[{"secondary_y": True}], [{"secondary_y": True}]],
        fig_production=fig_gas_production,
        fig_consumption=fig_gas_consumption,
        save_fig=False,
        file_name="chap_01_yearly_gas_consumption_production_combined_graph",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )

# Show figure
fig_gas_consumption_production_combine.show()

#### Financial Markets: Oil and Gas OI over time

In [ ]:
file_name = r"chap_06_prices_oi_df.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
prices_oi_df = pd.read_excel(full_file_path, index_col=0)

In [ ]:
variables = ["Oil"]
secondary_y_variables = ["Oil Open Interest USD"]

start_year = prices_oi_df.index.min().year
end_year = prices_oi_df.index.max().year

title = f"Weekly oil open interest as total number of contracts and in USD over the time: {start_year} - {end_year}"
x_axis_title = "Time"
y_axis_title = "Open interest (number of contracts)"
secondary_y_axis_title = "Open interest (in USD)"

custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(variables)+1)/2),
    lightening_factor=0.8,
)
# Extract only the hex color codes from the created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
oi_usd_color = "black"  # Fixed color for the "World" category

# Create a mapping for the colors
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(variables)}
color_mapping["Oil Open Interest USD"] = oi_usd_color

fig_oil_oi = data_graphing_instance.get_fig_open_interest(
        data=prices_oi_df,
        variables=variables,
        secondary_y_variables=secondary_y_variables,
        title=title,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        secondary_y_axis_title=secondary_y_axis_title,
        color_mapping_dict=color_mapping,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        save_fig=False,
        file_name="chap_06_weekly_oil_open_interest",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_oil_oi.show()

In [ ]:
variables = ["Gas"]
secondary_y_variables = ["Gas Open Interest USD"]


title = f"Weekly gas open interest as total number of contracts and in USD over the time: {start_year} - {end_year}"
x_axis_title = "Time"
y_axis_title = "Open interest (number of contracts)"
secondary_y_axis_title = "Open interest (in USD)"

custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(variables)+1)/2),
    lightening_factor=0.8,
)
# Extract only the hex color codes from the created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
oi_usd_color = "black"  # Fixed color for the "World" category

# Create a mapping for the colors
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(variables)}
color_mapping["Gas Open Interest USD"] = oi_usd_color

fig_gas_oi = data_graphing_instance.get_fig_open_interest(
        data=prices_oi_df,
        variables=variables,
        secondary_y_variables=secondary_y_variables,
        title=title,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        secondary_y_axis_title=secondary_y_axis_title,
        color_mapping_dict=color_mapping,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        save_fig=False,
        file_name="chap_06_weekly_gas_open_interest",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_gas_oi.show()

In [ ]:
subplot_titles=(f"Weekly oil open interest as total number of contracts and in USD over the time: {prices_oi_df.index.min().strftime('%Y')} - {prices_oi_df.index.max().strftime('%Y')}",
                f"Weekly gas open interest as total number of contracts and in USD over the time: {prices_oi_df.index.min().strftime('%Y')} - {prices_oi_df.index.max().strftime('%Y')}")
start_year = fig_gas_oi.data[0].x[0]
end_year = fig_gas_oi.data[0].x[-1]
title = f"Weekly open interest of oil and gas products over the time: {pd.Timestamp(start_year).strftime('%Y')} - {pd.Timestamp(end_year).strftime('%Y')}"
x_axis_title = "Time"

fig_gas_oil_open_interest_combine = data_graphing_instance.get_combined_open_interest_graph(
        subplot_titles=subplot_titles,
        title="",
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        x_axis_title=x_axis_title,
        rows=2,
        cols=1,
        shared_xaxes=False,
        vertical_spacing=0.25,
        specs=[[{"secondary_y": True}], [{"secondary_y": True}]],
        fig_oil_oi=fig_oil_oi,
        fig_gas_oi=fig_gas_oi,
        save_fig=False,
        file_name="chap_06_weekly_open_interest_oil_gas_combined_graph.pdf",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_gas_oil_open_interest_combine.show()

#### Interest Rate Benchmarks - Absolute Levels

In [ ]:
# Load interest rate comparison data
file_name = r"chap_06_interest_rate_comparison_df.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
interest_rate_comparison_df = pd.read_excel(full_file_path, index_col=0)

data = interest_rate_comparison_df.copy()
variables = interest_rate_comparison_df.columns.tolist()
secondary_yaxis_variables = []
color_discrete_sequence = ["grey", "black", "#9b0a7d"]
title = f"Daily BIS Central Bank Policy Rate and 3M Interbank Rates over the time: {data.index[0].year} - {data.index[-1].year}"
x_axis_title = "Date"
y_axis_title = "Interest Rate (%)"
secondary_yaxis_title = ""
color_mapping = {
    'USD 3M': "grey",
    'EUR 3M': "black",
    'USD CBPR': "#9b0a7d",
    'EUR CBPR': "lightgrey",
}

# Generate figure
fig_interest_rate_comp = data_graphing_instance.get_fig_relationship_main_vars(
        data=data,
        variables=variables,
        secondary_y_variables=secondary_yaxis_variables,
        title="",
        secondary_y_axis_title=secondary_yaxis_title,
        color_discrete_sequence=color_discrete_sequence,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        color_mapping_dict=color_mapping,
        save_fig=False,
        file_name="chap_06_interest_rate_comparison_bis_cbpr_vs_3m_interbank",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )

# Show figure
fig_interest_rate_comp.show()

#### Interest Rate Benchmarks - Relative Levels

In [ ]:
# Load interest rate comparison data
file_name = r"chap_06_interest_rate_comparison_df.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
interest_rate_comparison_df = pd.read_excel(full_file_path, index_col=0)

# Construct the differentials
interest_rate_comparison_df["USD-EUR CBPR"] = interest_rate_comparison_df["USD CBPR"] - interest_rate_comparison_df["EUR CBPR"]
interest_rate_comparison_df["USD-EUR 3M"] = interest_rate_comparison_df["USD 3M"] - interest_rate_comparison_df["EUR 3M"]


variables = ["USD-EUR CBPR", "USD-EUR 3M"]
data = interest_rate_comparison_df.copy()[variables]
secondary_yaxis_variables = []
color_discrete_sequence = ["grey", "black", "#9b0a7d"]
title = f"Daily BIS Central Bank Policy Rate and 3M Interbank Rates differentials (USD-EUR) over the time: {data.index[0].year} - {data.index[-1].year}"
x_axis_title = "Date"
y_axis_title = "Interest Rate differential (%) (USD-EUR)"
secondary_yaxis_title = ""
color_mapping = {
    'USD-EUR CBPR': "black",
    'USD-EUR 3M': "#9b0a7d",
}

# Generate figure
fig_interest_rate_diffs_comp = data_graphing_instance.get_fig_relationship_main_vars(
        data=data,
        variables=variables,
        secondary_y_variables=secondary_yaxis_variables,
        title="",
        secondary_y_axis_title=secondary_yaxis_title,
        color_discrete_sequence=color_discrete_sequence,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        color_mapping_dict=color_mapping,
        save_fig=False,
        file_name="chap_06_interest_rate_comparison_bis_cbpr_vs_3m_interbank_diffs",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )

# Show figure
fig_interest_rate_diffs_comp.show()

#### Main variables distributions (raw data - normalized)

In [ ]:
# Load spot exchange rate data
file_name = r"chap_06_spot_exchange_rate_data_df.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
spot_exchange_rate_data_df_normed = pd.read_excel(full_file_path, index_col=0)
# Normalize the data
spot_exchange_rate_data_df_normed = (spot_exchange_rate_data_df_normed - spot_exchange_rate_data_df_normed.mean()) / spot_exchange_rate_data_df_normed.std()

start_year = spot_exchange_rate_data_df_normed.index.min().strftime('%Y')
end_year = spot_exchange_rate_data_df_normed.index.max().strftime('%Y')
color_mapping_dict = {
    'EUR/USD': 'grey',
    "WTI Oil": 'black',
    "Nat Gas": 'lightgrey'
}
title=f"Normalized daily EUR/USD spot exchange rate, oil and gas over the time range: {start_year} - {end_year}"
x_axis_title="Daily Observations (Normalized)"
y_axis_title="Probability density"

# Generate figure
fig = data_graphing_instance.get_fig_histogram(
    data=spot_exchange_rate_data_df_normed,
    variables=spot_exchange_rate_data_df_normed.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    color_discrete_sequence=CAU_COLOR_SCALE,
    color_mapping_dict=color_mapping_dict,
    histnorm="probability density",
    draw_vertical_line_at_0=True,
    showlegend=False,
    save_fig=False,
    file_path=FIGURES_PATH,
    file_name="chap_06_raw_data_normalized_histogram",
    margin_dict=dict(
        l=20,  # Left margin
        r=20,  # Right margin
        t=100,  # Top margin
        b=10   # Bottom margin
    )
)

# Show figure
fig.show()

#### Main variables distributions (log first differences)

In [ ]:
# Load spot exchange rate data
file_name = r"chap_06_normed_histogram_data_log_first_differences.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
log_diff_spot_exchange_rate_data_df = pd.read_excel(full_file_path, index_col=0)

start_year = log_diff_spot_exchange_rate_data_df.index.min().strftime('%Y')
end_year = log_diff_spot_exchange_rate_data_df.index.max().strftime('%Y')
color_mapping_dict = {
    'EUR/USD': 'grey',
    "WTI Oil": 'black',
    "Nat Gas": 'lightgrey'
}
title=f"Daily log first differences of EUR/USD spot exchange rate, oil and gas over the time range: {start_year} - {end_year}",
x_axis_title="Daily observations (log first differences)"
y_axis_title="Probability density"

# Generate figure
fig = data_graphing_instance.get_fig_histogram(
    data=log_diff_spot_exchange_rate_data_df,
    variables=log_diff_spot_exchange_rate_data_df.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    color_discrete_sequence=CAU_COLOR_SCALE,
    color_mapping_dict=color_mapping_dict,
    histnorm="probability density",
    draw_vertical_line_at_0=True,
    showlegend=False,
    save_fig=False,
    file_path=FIGURES_PATH,
    file_name="chap_06_log_first_diff_histogram",
    margin_dict=dict(
        l=20,  # Left margin
        r=20,  # Right margin
        t=100,  # Top margin
        b=10   # Bottom margin
    )
)

# Show figure
fig.show()

#### Tests for Normality (raw data)

In [ ]:
# Load normality test results
file_name = r"chap_06_norm_test_raw_series.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
normality_test_results = pd.read_excel(full_file_path, index_col=0)
print(normality_test_results)

#### Tests for Normality (log first differences)

In [ ]:
# Load normality test results
file_name = r"chap_06_norm_test_log_diff.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
normality_test_results = pd.read_excel(full_file_path, index_col=0)
print(normality_test_results)

#### Tests for Stationarity - ADF Tests (raw data)

In [ ]:
# Load ADF test results
file_name = r"chap_06_adf_test_raw_series.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
adf_test_df = pd.read_excel(full_file_path, index_col=0)
print(adf_test_df)

#### Tests for Stationarity - ADF Tests (log first differences)

In [ ]:
# Load ADF test results
file_name = r"chap_06_adf_test_log_diff.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
adf_test_df = pd.read_excel(full_file_path, index_col=0)
print(adf_test_df)

#### Tests for Cointegration (raw data)

In [ ]:
# Load cointegration test results
file_name = r"chap_06_cointegration_test_raw_series.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
cointegration_test_df = pd.read_excel(full_file_path, index_col=0)
print(cointegration_test_df)

#### Tests for Cointegration (log differences)

In [ ]:
# Load cointegration test results
file_name = r"chap_06_cointegration_test_log_diff.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
cointegration_test_df = pd.read_excel(full_file_path, index_col=0)
print(cointegration_test_df)

#### Tests for Autocorrelation (raw data)

In [ ]:
# Load spot exchange rate data
file_name = r"chap_06_spot_exchange_rate_data_df.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
spot_exchange_rate_data_df = pd.read_excel(full_file_path, index_col=0)

title = f"ACF values for daily observations of the EUR/USD spot exchange rate, oil and gas over the time: {spot_exchange_rate_data_df.index.min().strftime('%Y')} - {spot_exchange_rate_data_df.index.max().strftime('%Y')}"
x_axis_title = "Lags"
y_axis_title = "ACF value"

# Generate ACF figure
acf_fig = data_graphing_instance.get_fig_acf(
    data=spot_exchange_rate_data_df,
    variables=spot_exchange_rate_data_df.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    save_fig=False,
    file_name="chap_06_acf_plot_raw_series",
    file_path=FIGURES_PATH,
    nlags=30
    )

# Show figure
acf_fig.show()

#### Tests for Autocorrelation (log first differences)

In [ ]:
# Transform to log values and take first difference
spot_exchange_rate_data_df_log_diff = np.log(spot_exchange_rate_data_df).diff().dropna()

title = f"ACF values for daily observations (log first differences) of the spot exchange rate EUR/USD, oil and gas over the time: {spot_exchange_rate_data_df_log_diff.index.min().strftime('%Y')} - {spot_exchange_rate_data_df_log_diff.index.max().strftime('%Y')}"
x_axis_title = "Lags"
y_axis_title = "ACF value"

# Generate ACF figure
acf_fig = data_graphing_instance.get_fig_acf(
    data=spot_exchange_rate_data_df_log_diff,
    variables=spot_exchange_rate_data_df_log_diff.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    save_fig=False,
    file_name="chap_06_acf_plot_log_diff",
    file_path=FIGURES_PATH,
    nlags=30
    )

# Show figure
acf_fig.show()

#### Tests for Partial Autocorrelation (raw data)

In [ ]:
# Load spot exchange rate data
title = f"PACF values for daily observations of the spot exchange rate EUR/USD, oil and gas over the time: {spot_exchange_rate_data_df.index.min().strftime('%Y')} - {spot_exchange_rate_data_df.index.max().strftime('%Y')}"
x_axis_title = "Lags"
y_axis_title = "PACF value"

# Generate PACF figure
pacf_fig = data_graphing_instance.get_fig_pacf(
    data=spot_exchange_rate_data_df,
    variables=spot_exchange_rate_data_df.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    save_fig=False,
    file_name="chap_06_pacf_plot_raw_series",
    file_path=FIGURES_PATH,
    nlags=30
    )

# Show figure
pacf_fig.show()

#### Tests for Partial Autocorrelation (log first differences)

In [ ]:
# Transform to log values and take first difference
title = f"PACF values for daily observations (log first differences) of the spot exchange rate EUR/USD, oil and gas over the time: {spot_exchange_rate_data_df_log_diff.index.min().strftime('%Y')} - {spot_exchange_rate_data_df_log_diff.index.max().strftime('%Y')}"
x_axis_title = "Lags"
y_axis_title = "PACF value"

# Generate PACF figure
pacf_fig = data_graphing_instance.get_fig_pacf(
    data=spot_exchange_rate_data_df_log_diff,
    variables=spot_exchange_rate_data_df_log_diff.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    save_fig=False,
    file_name="chap_06_pacf_plot_log_diff",
    file_path=FIGURES_PATH,
    nlags=30
    )

# Show figure
pacf_fig.show()

#### Granger Causality Tests - EUR/USD and oil (raw data)

In [ ]:
# Load Granger causality test results for oil
file_name = r"chap_06_granger_causality_test_oil_raw_series.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
granger_test_result_df_oil = pd.read_excel(full_file_path)

variables = granger_test_result_df_oil['Metric'].unique()
secondary_y_variables = ["p-value"]
color_mapping_dict = {
    'p-value': 'red',
    "ssr_ftest": 'darkgrey',
    "ssr_chi2test": 'grey',
    "lrtest": 'black',
    "params_ftest": 'lightgrey',
}
title=f"Granger causality test results testing granger causality of daily observations of oil for EUR/USD over the time: {spot_exchange_rate_data_df.index.min().strftime('%Y')} - {spot_exchange_rate_data_df.index.max().strftime('%Y')}"
secondary_y_axis_title="p-value"

# Generate figure
granger_causality_test_plot = data_graphing_instance.plot_granger_test_results(
    data=granger_test_result_df_oil,
    variables=variables,
    secondary_y_variables=secondary_y_variables,
    color_discrete_sequence=["#9b0a7d", "grey", "black", "darkgrey", "lightgrey"],
    title="",
    secondary_y_axis_title=secondary_y_axis_title,
    x_axis_title="Lag",
    y_axis_title="Test-Statistic",
    color_mapping_dict=color_mapping_dict,
    significance_level=0.05,
    margin_dict=dict(
            l=20,  # Left margin
            r=20,  # Right margin
            t=50,  # Top margin
            b=10   # Bottom margin
            ),
    showlegend=False,
    save_fig=False,
    file_name="chap_06_granger_causality_test_oil_raw_series",
    file_path=FIGURES_PATH
    )

# Show figure
granger_causality_test_plot.show()

#### Granger Causality Tests - EUR/USD and gas (raw data)

In [ ]:
# Load Granger causality test results for gas
file_name = r"chap_06_granger_causality_test_gas_raw_series.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
granger_test_result_df_gas = pd.read_excel(full_file_path)

variables = granger_test_result_df_gas['Metric'].unique()
secondary_y_variables = ["p-value"]
color_mapping_dict = {
    'p-value': 'red',
    "ssr_ftest": 'darkgrey',
    "ssr_chi2test": 'grey',
    "lrtest": 'black',
    "params_ftest": 'lightgrey',
}
title=f"Granger causality test results testing granger causality of daily observations of oil for EUR/USD over the time: {spot_exchange_rate_data_df.index.min().strftime('%Y')} - {spot_exchange_rate_data_df.index.max().strftime('%Y')}"
secondary_y_axis_title="p-value"

# Generate figure
granger_causality_test_plot = data_graphing_instance.plot_granger_test_results(
    data=granger_test_result_df_gas,
    variables=variables,
    secondary_y_variables=secondary_y_variables,
    color_discrete_sequence=["#9b0a7d", "grey", "black", "darkgrey", "lightgrey"],
    title="",
    secondary_y_axis_title=secondary_y_axis_title,
    x_axis_title="Lag",
    y_axis_title="Test-Statistic",
    color_mapping_dict=color_mapping_dict,
    significance_level=0.05,
    margin_dict=dict(
            l=20,  # Left margin
            r=20,  # Right margin
            t=50,  # Top margin
            b=10   # Bottom margin
            ),
    showlegend=False,
    save_fig=False,
    file_name="chap_06_granger_causality_test_oil_raw_series",
    file_path=FIGURES_PATH
    )

# Show figure
granger_causality_test_plot.show()

#### Granger Causality Tests - EUR/USD and oil (log first differences)

In [ ]:
# Load Granger causality test results for oil (log first differences)
file_name = r"chap_06_granger_causality_test_oil_log_diff.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
granger_test_result_df_oil = pd.read_excel(full_file_path)

variables = granger_test_result_df_oil['Metric'].unique()
secondary_y_variables = ["p-value"]
color_mapping_dict = {
    'p-value': 'red',
    "ssr_ftest": 'darkgrey',
    "ssr_chi2test": 'grey',
    "lrtest": 'black',
    "params_ftest": 'lightgrey',
}
title=f"Granger causality test results testing granger causality of daily observations (log first differences) of oil for EUR/USD over the time: {spot_exchange_rate_data_df_log_diff.index.min().strftime('%Y')} - {spot_exchange_rate_data_df_log_diff.index.max().strftime('%Y')}"
secondary_y_axis_title="p-value"

# Generate figure
granger_causality_test_plot = data_graphing_instance.plot_granger_test_results(
    data=granger_test_result_df_oil,
    variables=variables,
    secondary_y_variables=secondary_y_variables,
    color_discrete_sequence=["#9b0a7d", "grey", "black", "darkgrey", "lightgrey"],
    title="",
    secondary_y_axis_title=secondary_y_axis_title,
    x_axis_title="Lag",
    y_axis_title="Test-Statistic",
    color_mapping_dict=color_mapping_dict,
    significance_level=0.05,
    margin_dict=dict(
            l=20,  # Left margin
            r=20,  # Right margin
            t=50,  # Top margin
            b=10   # Bottom margin
            ),
    showlegend=False,
    save_fig=False,
    file_name="chap_06_granger_causality_test_oil_log_diff",
    file_path=FIGURES_PATH
    )

# Show figure
granger_causality_test_plot.show()

#### Granger Causality Tests - EUR/USD and gas (log first differences)

In [ ]:
# Load Granger causality test results for gas (log first differences)
file_name = r"chap_06_granger_causality_test_gas_log_diff.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
granger_test_result_df_gas = pd.read_excel(full_file_path)

variables = granger_test_result_df_gas['Metric'].unique()
secondary_y_variables = ["p-value"]
title=f"Granger causality test results testing granger causality of daily observations (log first differences) of gas for EUR/USD over the time: {spot_exchange_rate_data_df_log_diff.index.min().strftime('%Y')} - {spot_exchange_rate_data_df_log_diff.index.max().strftime('%Y')}"

# Generate figure
granger_causality_test_plot = data_graphing_instance.plot_granger_test_results(
    data=granger_test_result_df_gas,
    variables=variables,
    secondary_y_variables=secondary_y_variables,
    color_discrete_sequence=["#9b0a7d", "grey", "black", "darkgrey", "lightgrey"],
    title="",
    secondary_y_axis_title="p-value",
    x_axis_title="Lag",
    y_axis_title="Test-Statistic",
    color_mapping_dict=color_mapping_dict,
    significance_level=0.05,
    margin_dict=dict(
            l=20,  # Left margin
            r=20,  # Right margin
            t=50,  # Top margin
            b=10   # Bottom margin
            ),
    showlegend=False,
    save_fig=False,
    file_name="chap_06_granger_causality_test_gas_log_diff",
    file_path=FIGURES_PATH
    )

# Show figure
granger_causality_test_plot.show()